In [1]:
import psycopg2
import json

In [3]:
# Open the JSON file
with open('/Users/xiaosuhu/Documents/Python/RAG_fNIRS/fNIRS_abstract_content.json', 'r') as file:
    data = json.load(file)  # Parse JSON into a Python dictionary or list



In [7]:
data['pages'] = [page.replace('\n', '') for page in data['pages']]
print(data['pages'][0])

Mapping neural correlates of biological motion perception in autistic children using high -density diffuse optical tomography  Dalin Yang1, Alexandra M. Svoboda1, Tessa G. George1, Patricia K. Mansfield1, Muriah D. Wheelock1, Mariel L. Schroeder1, Sean M. Rafferty1, Arefeh Sherafati1, Kalyan Tripathy1, Tracy Burns -Yocum1, Elizabeth Forsen1, John R. Pruett, Jr2, Natasha M. Marrus2, Joseph P. Culver1, John N. Constantino3, Adam T. Eggebrecht11Washington University School of Medicine,  Mallinckrodt Institute of Radiology, St. Louis, Missouri 63110, USA . 2Washington University School of Medicine, Department of Psychiatry, St. Louis, Missouri, 63110, USA. 3Emory University School of Medicine, Department of Psychiatry, Atlanta, Georgia 30322, USA.  Presenting Author email address: yangd@wustl.edu  Abstract:  We used high-density diffuse optical tomograph y (HD-DOT ) to image brain function in 102 school -age children with and without autism while  they performed a passive biological motion

In [11]:
# Rearrange the data into the desired format
formatted_data = [
    {"title": f"Page {i+1}", "content": page}
    for i, page in enumerate(data['pages'])
]
len(formatted_data)

387

In [12]:
def connect_db():
    return psycopg2.connect(
        host='localhost',
        database = 'postgres',
        user = 'postgres',
        password = 'password',
        port = '6000'
    )

In [13]:
conn = connect_db()
cur = conn.cursor()
cur.execute(""" 
    CREATE TABLE IF NOT EXISTS documents (
            id SERIAL PRIMARY KEY,
            title TEXT,
            content TEXT,
            embedding VECTOR(768)
            );
""")
conn.commit()
cur.close()
conn.close()

In [16]:
conn = connect_db()
cur = conn.cursor()

# use the port at which your ollama service is running.
for doc in formatted_data:
    cur.execute("""
        INSERT INTO public.documents (title, content, embedding)
        VALUES (
            %(title)s,
            %(content)s,
            ollama_embed('nomic-embed-text', concat(%(title)s, ' - ', %(content)s), _host=>'http://ollama_fnirs:11434')
        )
    """, doc)

conn.commit()
cur.close()
conn.close()

In [17]:
conn = connect_db()
cur = conn.cursor()
    
cur.execute("""
    SELECT title, content, vector_dims(embedding) 
    FROM documents
    LIMIT 5;
""")

rows = cur.fetchall()
for row in rows:
    print(f"Title: {row[0]}, Content: {row[1]}, Embedding Dimensions: {row[2]}")

cur.close()
conn.close()

Title: Page 1, Content: Mapping neural correlates of biological motion perception in autistic children using high -density diffuse optical tomography  Dalin Yang1, Alexandra M. Svoboda1, Tessa G. George1, Patricia K. Mansfield1, Muriah D. Wheelock1, Mariel L. Schroeder1, Sean M. Rafferty1, Arefeh Sherafati1, Kalyan Tripathy1, Tracy Burns -Yocum1, Elizabeth Forsen1, John R. Pruett, Jr2, Natasha M. Marrus2, Joseph P. Culver1, John N. Constantino3, Adam T. Eggebrecht11Washington University School of Medicine,  Mallinckrodt Institute of Radiology, St. Louis, Missouri 63110, USA . 2Washington University School of Medicine, Department of Psychiatry, St. Louis, Missouri, 63110, USA. 3Emory University School of Medicine, Department of Psychiatry, Atlanta, Georgia 30322, USA.  Presenting Author email address: yangd@wustl.edu  Abstract:  We used high-density diffuse optical tomograph y (HD-DOT ) to image brain function in 102 school -age children with and without autism while  they performed a p